In [1]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, ForeignKey
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import relationship

In [2]:
import numpy as np

In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///InPTA.db', echo = True)

In [4]:
Base = declarative_base()

In [5]:
class Member(Base):
    __tablename__ = 'members'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(100))
    username = Column(String(45))
    email = Column(String(100))

    def __repr__(self):
        return "<Member(id='%s', name='%s', username='%s')>" % (self.id, self.name, self.username)

In [6]:
def hms_to_rad(raj):
    hh, mm, ss = raj.split(':')
    rad = (float(hh) + float(mm)/60. + float(ss)/3600.)*np.pi/24.
    return rad

def dms_to_rad(decj):
    dd, mm, ss = decj.split(':')
    sign = np.sign(float(dd))
    rad = sign*(sign*float(dd) + float(mm)/60. + float(ss)/3600.)*np.pi/180.
    return rad

In [7]:
class Source(Base):
    __tablename__ = 'sources'
    
    jname = Column(String(45), primary_key=True)
    srctype = Column(String(10))
    raj = Column(Float)
    decj = Column(Float)
    
    def __init__(self, jname, srctype, raj, decj):
        self.jname = jname
        self.srctype = srctype
        self.raj = hms_to_rad(raj)
        self.decj = dms_to_rad(decj)
        
    def __repr__(self):
        return "<Source(JName='%s', Type='%s', RA='%s', DEC='%s')>" % (self.jname, self.type, self.raj, self.decj)

In [8]:
class Telescope(Base):
    __tablename__ = 'telescopes'
    
    id = Column(String(10), primary_key=True)
    name = Column(String(45))
    lat = Column(Float)
    long = Column(Float)

    def __repr__(self):
        return "<Telescope('%s')>" % (self.id)

In [9]:
class Proposal(Base):
    __tablename__ = 'proposals'
    
    number = Column(String(10), primary_key=True)
    cycle_number = Column(Integer)
    title = Column(String(500))
    telescope_ID = Column(String(10), ForeignKey("telescopes.id"))
    pi_id = Column(Integer, ForeignKey("members.id"))
    file = Column(String(500))
    
    telescope = relationship("Telescope")
    PI = relationship("Member")

    def __repr__(self):
        return "<Proposal(number='%s',title='%s')>" % (self.number, self.title)

In [10]:
Base.metadata.create_all(engine)

2020-08-24 12:13:36,267 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-08-24 12:13:36,276 INFO sqlalchemy.engine.base.Engine ()
2020-08-24 12:13:36,284 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-08-24 12:13:36,290 INFO sqlalchemy.engine.base.Engine ()
2020-08-24 12:13:36,296 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("members")
2020-08-24 12:13:36,310 INFO sqlalchemy.engine.base.Engine ()
2020-08-24 12:13:36,317 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("sources")
2020-08-24 12:13:36,324 INFO sqlalchemy.engine.base.Engine ()
2020-08-24 12:13:36,329 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("telescopes")
2020-08-24 12:13:36,331 INFO sqlalchemy.engine.base.Engine ()
2020-08-24 12:13:36,335 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("proposals")
2020-08-24 12:13:36,337 INFO sqlalchemy.engine.base.Engine ()


In [11]:
Session = sessionmaker(bind=engine)
session = Session()

In [12]:
testsrc = Source(jname="J0002+6216", raj="00:02:58.17", decj="+62:16:09.4", srctype="PSR")
session.add(testsrc)
session.commit()

2020-08-24 12:13:39,575 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-24 12:13:39,583 INFO sqlalchemy.engine.base.Engine INSERT INTO sources (jname, srctype, raj, decj) VALUES (?, ?, ?, ?)
2020-08-24 12:13:39,587 INFO sqlalchemy.engine.base.Engine ('J0002+6216', 'PSR', 0.0064784440172464515, 1.0868039200611601)
2020-08-24 12:13:39,594 INFO sqlalchemy.engine.base.Engine COMMIT


In [13]:
gmrt = Telescope(id='GMRT', name='Giant Meterwave Radio Telescope', lat=0.33329486135, long=1.29241146296)
session.add(gmrt)
session.commit()

2020-08-24 12:13:48,580 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-24 12:13:48,588 INFO sqlalchemy.engine.base.Engine INSERT INTO telescopes (id, name, lat, long) VALUES (?, ?, ?, ?)
2020-08-24 12:13:48,593 INFO sqlalchemy.engine.base.Engine ('GMRT', 'Giant Meterwave Radio Telescope', 0.33329486135, 1.29241146296)
2020-08-24 12:13:48,599 INFO sqlalchemy.engine.base.Engine COMMIT


In [14]:
testmember = Member(name="Test User", username="testuser", email="testusersassd@gmail.com")
session.add(testmember)
session.commit()

2020-08-24 12:13:52,083 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-24 12:13:52,087 INFO sqlalchemy.engine.base.Engine INSERT INTO members (name, username, email) VALUES (?, ?, ?)
2020-08-24 12:13:52,088 INFO sqlalchemy.engine.base.Engine ('Test User', 'testuser', 'testusersassd@gmail.com')
2020-08-24 12:13:52,090 INFO sqlalchemy.engine.base.Engine COMMIT


In [ ]:
session.commit()